In [318]:
import numpy as np
from sympy import *

In [319]:
theta_joy1 = symbols('th_j1')
theta_joy2 = symbols('th_j2')
theta_joy3 = symbols('th_j3')
theta_joy4 = symbols('th_j4')

L_joy_shoulder_from_body = symbols('L_joy_shoulder_from_body')
L_joy_shoulder_x = symbols('L_joy_shoulder_x')
L_joy_shoulder_y = symbols('L_joy_shoulder_y')
L_joy_arm = symbols('L_joy_arm')
L_joy_forearm = symbols('L_joy_forearm')
L_joy_hand = symbols('L_joy_hand')
L_sat_shoulder_from_body = symbols('L_sat_shoulder_from_body')
L_sat_forearm = symbols('L_sat_forearm')
L_sat_arm = symbols('L_sat_arm')

joy_right_shoulder = Matrix([0, -L_joy_shoulder_from_body-L_joy_shoulder_y, 0])
sat_right_shoulder = Matrix([0, -L_sat_shoulder_from_body, 0])

In [320]:
def HTM(axis, theta, displacement):
    R = eye(3)
    if(axis == 'x'):
        R = Matrix([
            [1, 0, 0],
            [0, cos(theta), -sin(theta)],
            [0, sin(theta), cos(theta)],
        ]
    )
    if(axis == 'y'):
        R = Matrix([
            [cos(theta), 0, sin(theta)],
            [0, 1, 0],
            [-sin(theta), 0, cos(theta)],
        ]
    )
    if(axis == 'z'):
        R = Matrix([
            [cos(theta), -sin(theta), 0],
            [sin(theta), cos(theta), 0],
            [0, 0, 1]
        ]
    )
    return R.row_join(Matrix(displacement)).col_join(Matrix([0, 0, 0, 1]).T)

H_shoulder_1 = HTM('z', theta_joy1, [0, -L_joy_shoulder_from_body, 0])
H_shoulder_2 = HTM('x', theta_joy2, [0, 0, 0])
H_shoulder_3 = HTM('y', theta_joy3, [L_joy_shoulder_x, -L_joy_shoulder_y, 0])
H_elbow = HTM('y', -theta_joy4, [0, 0, -L_joy_arm])
H_end = HTM('', 0, [0, L_joy_hand, -L_joy_forearm])

H_end_0 = simplify(Matrix(H_shoulder_1@H_shoulder_2@H_shoulder_3@H_elbow@H_end))

print(ccode(H_end_0, assign_to=MatrixSymbol('joy_eff', 4, 4)))

joy_eff[0] = -sin(th_j1)*sin(th_j2)*sin(th_j3 - th_j4) + cos(th_j1)*cos(th_j3 - th_j4);
joy_eff[1] = -sin(th_j1)*cos(th_j2);
joy_eff[2] = sin(th_j1)*sin(th_j2)*cos(th_j3 - th_j4) + sin(th_j3 - th_j4)*cos(th_j1);
joy_eff[3] = -L_joy_arm*(sin(th_j1)*sin(th_j2)*cos(th_j3) + sin(th_j3)*cos(th_j1)) - L_joy_forearm*(sin(th_j1)*sin(th_j2)*cos(th_j3 - th_j4) + sin(th_j3 - th_j4)*cos(th_j1)) - L_joy_hand*sin(th_j1)*cos(th_j2) + L_joy_shoulder_x*cos(th_j1) + L_joy_shoulder_y*sin(th_j1)*cos(th_j2);
joy_eff[4] = sin(th_j1)*cos(th_j3 - th_j4) + sin(th_j2)*sin(th_j3 - th_j4)*cos(th_j1);
joy_eff[5] = cos(th_j1)*cos(th_j2);
joy_eff[6] = sin(th_j1)*sin(th_j3 - th_j4) - sin(th_j2)*cos(th_j1)*cos(th_j3 - th_j4);
joy_eff[7] = -L_joy_arm*(sin(th_j1)*sin(th_j3) - sin(th_j2)*cos(th_j1)*cos(th_j3)) - L_joy_forearm*(sin(th_j1)*sin(th_j3 - th_j4) - sin(th_j2)*cos(th_j1)*cos(th_j3 - th_j4)) + L_joy_hand*cos(th_j1)*cos(th_j2) - L_joy_shoulder_from_body + L_joy_shoulder_x*sin(th_j1) - L_joy_shoulder_y*cos(th_j1)*c

In [321]:
Rxx, Rxy, Rxz, dx = symbols('joy_eff[0] joy_eff[1] joy_eff[2] joy_eff[3]')
Ryx, Ryy, Ryz, dy = symbols('joy_eff[4] joy_eff[5] joy_eff[6] joy_eff[7]')
Rzx, Rzy, Rzz, dz = symbols('joy_eff[8] joy_eff[9] joy_eff[10] joy_eff[11]')

p_joy_end = Matrix([dx, dy, dz])
yaxis_joy_end = Matrix([Rxy, Ryy, Rzy])
zaxis_joy_end = Matrix([Rxz, Ryz, Rzz])

p_joy_elb = p_joy_end - L_joy_forearm * -zaxis_joy_end
p_shoulder_zero = Matrix([L_joy_shoulder_x, -L_joy_shoulder_from_body - L_joy_shoulder_y + L_joy_hand, 0])

sat_elb_dir = p_joy_elb - p_shoulder_zero
print(ccode(sat_elb_dir, assign_to=MatrixSymbol('sat_elb_dir', 3, 1)))

sat_elb_dir[0] = L_joy_forearm*joy_eff[2] - L_joy_shoulder_x + joy_eff[3];
sat_elb_dir[1] = L_joy_forearm*joy_eff[6] - L_joy_hand + L_joy_shoulder_from_body + L_joy_shoulder_y + joy_eff[7];
sat_elb_dir[2] = L_joy_forearm*joy_eff[10] + joy_eff[11];


In [322]:
def norm3d(v):
    mag = sqrt(v[0]**2 + v[1]**2 + v[2]**2)
    return v/mag

sat_elb_dir0, sat_elb_dir1, sat_elb_dir2 = symbols('sat_elb_dir[0] sat_elb_dir[1] sat_elb_dir[2]')
sat_elb_dir_sym = Matrix([sat_elb_dir0, sat_elb_dir1, sat_elb_dir2])
p_sat_elb = L_sat_arm * norm3d(sat_elb_dir_sym)

Rz = simplify(-norm3d(sat_elb_dir_sym))
print(ccode(Rz, assign_to=MatrixSymbol('Rz', 3, 1)))

Rz[0] = -sat_elb_dir[0]/sqrt(pow(sat_elb_dir[0], 2) + pow(sat_elb_dir[1], 2) + pow(sat_elb_dir[2], 2));
Rz[1] = -sat_elb_dir[1]/sqrt(pow(sat_elb_dir[0], 2) + pow(sat_elb_dir[1], 2) + pow(sat_elb_dir[2], 2));
Rz[2] = -sat_elb_dir[2]/sqrt(pow(sat_elb_dir[0], 2) + pow(sat_elb_dir[1], 2) + pow(sat_elb_dir[2], 2));


In [323]:
Rz0, Rz1, Rz2 = symbols('Rz[0] Rz[1] Rz[2]')
Rz_sym = Matrix([Rz0, Rz1, Rz2])

Ry = simplify((yaxis_joy_end - Rz_sym*yaxis_joy_end.dot(Rz_sym)))
print(ccode(Ry, assign_to=MatrixSymbol('Ry', 3, 1)))

#and then normalize later

Ry[0] = -Rz[0]*(Rz[0]*joy_eff[1] + Rz[1]*joy_eff[5] + Rz[2]*joy_eff[9]) + joy_eff[1];
Ry[1] = -Rz[1]*(Rz[0]*joy_eff[1] + Rz[1]*joy_eff[5] + Rz[2]*joy_eff[9]) + joy_eff[5];
Ry[2] = -Rz[2]*(Rz[0]*joy_eff[1] + Rz[1]*joy_eff[5] + Rz[2]*joy_eff[9]) + joy_eff[9];


In [324]:
Ry0, Ry1, Ry2 = symbols('Ry[0] Ry[1] Ry[2]')
Ry_sym = Matrix([Ry0, Ry1, Ry2])

Rx = (Ry_sym.cross(Rz_sym))
print(ccode(Rx, assign_to=MatrixSymbol('Rx', 3, 1)))

Rx0, Rx1, Rx2 = symbols('Rx[0] Rx[1] Rx[2]')
Rx_sym = Matrix([Rx0, Rx1, Rx2])


Rx[0] = Ry[1]*Rz[2] - Ry[2]*Rz[1];
Rx[1] = -Ry[0]*Rz[2] + Ry[2]*Rz[0];
Rx[2] = Ry[0]*Rz[1] - Ry[1]*Rz[0];


In [325]:
v1 = zaxis_joy_end
v2 = Rz_sym
axis = Ry_sym

th_s4 = acos(v1.dot(v2))
print(ccode(simplify(th_s4)))

# print(ccode(simplify(v1.cross(v2).dot(axis))))


acos(Rz[0]*joy_eff[2] + Rz[1]*joy_eff[6] + Rz[2]*joy_eff[10])


In [326]:
R = Matrix([Rx_sym.T, Ry_sym.T, Rz_sym.T]).T
shift = Matrix([
    [0, 0, 1],
    [1, 0, 0],
    [0, 1, 0]
])

R = shift @ R

s5 = sqrt(1 - R[2,2]*R[2,2])

th_s1 = atan2(R[0,2], R[1,2]) - pi/2
th_s2 = atan2(R[2,2], s5)
th_s3 = atan2(-R[2,0], R[2,1])


def ang_wrap(num):
    return Mod(num + pi, 2*pi) - pi;

th_s1 = -ang_wrap(th_s1)
th_s2 = -ang_wrap(th_s2)
th_s3 = -ang_wrap(th_s3)

print(ccode(simplify(th_s1)))
print(ccode(simplify(th_s2)))
print(ccode(simplify(th_s3)))
# Vector3d sol1(-ang_wrap(theta4), -ang_wrap(theta5), -ang_wrap(theta6));

M_PI - (fmod(atan2(Rz[2], Rz[0]) + M_PI_2, 2*M_PI))
M_PI - (fmod(atan2(Rz[1], sqrt(1 - pow(Rz[1], 2))) + M_PI, 2*M_PI))
M_PI - (fmod(atan2(-Rx[1], Ry[1]) + M_PI, 2*M_PI))
